# REVIEW SUMMARIZER
## TRIPADVISOR: HOTELS

*   Esteban Ariza
*   Johan Giraldo
*   Mateo Valdes

## Prerequisites

In [1]:
%pip install transformers
%pip install torch
%pip install sentencepiece
%pip install rouge-score
%pip install evaluate

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=08d81ed32186507a800245ae6f92fd8932b2a55f6782b34662fdb8db0f565979
  Stored in directory: c:\users\ariza\appdata\local\pip\cache\wheels\9b\3d\39\09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge-score
Note: you may need to restart the kernel to use updated packages.
     ---------------------------------------- 72.9/72.9 kB 2.0 MB/s eta 0:00:00
     ---------------------------------------- 95.8/95.8 kB ? eta 0:00:00
     -------------------------------------- 132.3/132.3 kB 3.9 MB/s eta 0:00:00
     -------------------------------------- 441.9/441.9 kB 9.2 MB/s eta 0:00:00
 

In [2]:
import torch
import json
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
import pandas as pd
import csv
from rouge_score import rouge_scorer, scoring
import evaluate
from nltk.translate.bleu_score import sentence_bleu

In [3]:
COLUMNS_NAME = ['ORIGINAL_TEXT', 'SUMMARIZED_TEXT']

In [ ]:
try:
    writer = csv.DictWriter(open('summarized_reviews.csv', 'w', encoding='UTF8', newline=''), fieldnames=COLUMNS_NAME, delimiter=',', lineterminator='\r')
    writer.writeheader()
except IOError:
    print("I/O error")

In [ ]:
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def summarize(review):
    tokenized_text = tokenizer.encode('summarize: ' + review, return_tensors="pt").to(device)
    summary_ids = model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=100,
                                    early_stopping=True)
    row = {}
    row[COLUMNS_NAME[0]] = review
    row[COLUMNS_NAME[1]] = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    try:
        writer.writerow(row)
    except IOError:
                print("I/O error")
    print('Summarized: ' + row[COLUMNS_NAME[0]] + ' to: ' + row[COLUMNS_NAME[1]])

In [ ]:
data = pd.read_csv('tripadvisor_hotels_sustainables_v2.csv')
#data['REVIEW_TEXT'].apply(summarize)

In [4]:
predictions_df = pd.read_csv('summarized_reviews.csv')

In [5]:
rouge = evaluate.load('rouge')
predictions = predictions_df[COLUMNS_NAME[1]].tolist()
references = predictions_df[COLUMNS_NAME[0]].tolist()
results = rouge.compute(predictions=predictions, references=references)
print(results)

{'rouge1': 0.613452605751524, 'rouge2': 0.5577686141185489, 'rougeL': 0.565767682226821, 'rougeLsum': 0.5675337655033166}


In [20]:
def splitter(value):
    return value.split()

reference = list(map(splitter, predictions_df[COLUMNS_NAME[0]].tolist()))
candidates = list(map(splitter, predictions_df[COLUMNS_NAME[1]].tolist()))

# 1-gram:
def bleu(reference, candidates, weights=(0.25, 0.25, 0.25, 0.25)):
    result = 0;
    for candidate in candidates:
        result += sentence_bleu(reference, candidate, weights=weights)
    result = result / len(candidates)
    return result

print('BLEU: %f' %bleu(reference, candidates))
print('BLEU 1-gram: %f' %bleu(reference, candidates, (1, 0, 0, 0)))
print('BLEU 2-gram: %f' %bleu(reference, candidates, (0, 1, 0, 0)))
print('BLEU 3-gram: %f' %bleu(reference, candidates, (0, 0, 1, 0)))
print('BLEU 4-gram: %f' %bleu(reference, candidates, (0, 0, 0, 1)))

BLEU: 0.717735
BLEU 1-gram: 0.891337
BLEU 2-gram: 0.788820
BLEU 3-gram: 0.669039
BLEU 4-gram: 0.581243
